# APDFT Analysis



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from apdft_tools.utils import hartree_to_ev, get_multiplicity, _remove_dimer_outliers, _dimer_poly_pred, find_poly_min, add_energies_to_df_apdft
from apdft_tools.data import prepare_dfs
from apdft_tools.prediction import get_dimer_curve, get_apdft_refs

json_path = '/home/alex/Dropbox/keith/projects/apdft/apdft_tools/json-data/dimer-pyscf.apdft-data.posthf.json'
df_qc, df_apdft = prepare_dfs(json_path, get_CBS=False)

## Plotting bond lengths

In [2]:
system_label = 'be.h'
charge = 0  # Initial charge of the system.
excitation_level = 0
lambda_value = 1

basis_set = 'cc-pV5Z'
lambda_specific_atom = 0
n_points = 2
poly_order = 4
use_fin_diff = True

qm_color = '#577590'
apdft_color = '#F3722C'

In [3]:
df_qc_system = df_qc.query(
    'system == @system_label'
    '& charge == @charge'
    '& lambda_value == @lambda_value'
)
multiplicity = get_multiplicity(df_qc_system, excitation_level)
df_qc_system = df_qc_system.query('multiplicity == @multiplicity')
target_n_electrons = df_qc_system.iloc[0]['n_electrons']

bond_lengths_qc, energies_qc = get_dimer_curve(
    df_qc_system, lambda_value, use_fin_diff=False, apdft_order=None
)

if use_fin_diff:
    df_apdft_system = add_energies_to_df_apdft(
        df_qc, df_apdft
    )
    print(df_apdft_system)
    multiplicity = get_multiplicity(df_apdft_system, excitation_level)
    df_apdft_system = df_apdft_system.query('multiplicity == @multiplicity')
    bond_lengths_fin_diff, energies_fin_diff = get_dimer_curve(
        df_apdft_system, lambda_value, use_fin_diff=True, apdft_order=4
    )
    


###   FIGURE   ###
fig, ax = plt.subplots(1, 1, tight_layout=True)
fig.patch.set_facecolor('white')

ax.plot(
    bond_lengths_qc, bond_lengths_qc,
    marker='0', markersize=5,
    linewidth=0, color=qm_color
)
ax.plot(
    bond_lengths_fin_diff, bond_lengths_qc,
    marker='', markersize=5,
    linewidth=0.2, color=qm_color
)

ax.set_xlabel('Bond Length ($\AA$)')

ax.set_ylabel('Energy (Ha)')

      system atomic_numbers  charge  multiplicity  n_electrons qc_method  \
0       li.h         [3, 1]       0             3            4  UCCSD(T)   
1       li.h         [3, 1]       0             3            4  UCCSD(T)   
2       li.h         [3, 1]       0             3            4  UCCSD(T)   
3       li.h         [3, 1]       0             3            4  UCCSD(T)   
4       li.h         [3, 1]       0             3            4  UCCSD(T)   
...      ...            ...     ...           ...          ...       ...   
11699    f.h         [9, 1]       1             4            9  UCCSD(T)   
11700    f.h         [9, 1]       1             4            9  UCCSD(T)   
11701    f.h         [9, 1]       1             4            9  UCCSD(T)   
11702    f.h         [9, 1]       1             4            9  UCCSD(T)   
11703    f.h         [9, 1]       1             4            9  UCCSD(T)   

      basis_set lambda_range  finite_diff_delta  finite_diff_acc  \
0       cc-pV5Z    

AssertionError: 

## Polynomial fit to minimum

In [ ]:
remove_outliers = True
zscore_cutoff = 3.0

poly_bl_buffer = 0.1

idx_sort = np.argsort(bond_lengths_qc)
bond_lengths_qc = bond_lengths_qc[idx_sort]
e_qc = e_qc[idx_sort]

if remove_outliers:
    bond_lengths_qc, e_qc = _remove_dimer_outliers(
        bond_lengths_qc, e_qc
    )

e_min_idx = np.argmin(e_qc)
slice_start = e_min_idx - n_points
slice_end = e_min_idx + 1 + n_points
bond_lengths_for_fit = bond_lengths_qc[slice_start:slice_end]
e_for_fit = e_qc[slice_start:slice_end]

poly_coeffs = np.polyfit(bond_lengths_for_fit, e_for_fit, poly_order)
poly_bond_lengths = np.linspace(
    bond_lengths_for_fit[0]-poly_bl_buffer, bond_lengths_for_fit[-1]+poly_bl_buffer, num=100
)

poly_predictions = np.zeros(poly_bond_lengths.shape)
for i in range(len(poly_predictions)):
    poly_predictions[i] = _dimer_poly_pred(poly_bond_lengths[i], poly_coeffs)
    
bond_length_eq, e_eq = find_poly_min(poly_coeffs)

print(f'Equilibrium bond length: {bond_length_eq:.3f} Ang.')

###   FIGURE   ###
fig, ax = plt.subplots(1, 1, tight_layout=True)
fig.patch.set_facecolor('white')

ax.plot(
    bond_lengths_for_fit, e_for_fit,
    marker='o', markersize=5,
    linestyle='', color=qm_color
)
ax.plot(
    poly_bond_lengths, poly_predictions,
    marker='',
    linestyle='--', zorder=0, color='#F94144'
)

ax.set_xlabel('Bond Length ($\AA$)')

ax.set_ylabel('Energy (Ha)')
